In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.inspection import permutation_importance
from load_data import GetZeoliteTsv
import matplotlib.pyplot as plt
from sklearn import metrics
from scipy import stats
import seaborn as sns
import pandas as pd
import numpy as np
import os
np.random.seed(1)

In [2]:
#zeolite datafile exported from excel
zeolite_fname = "C:\\Users\\DrewX\\Documents\\Project-Roger-Dodger\\Python-ML\\zeolites database catagories V 2.txt"
#filename for datafile
zeolite_outfile = "ZeoX_Final_encodedx.tsv" 

In [3]:
#open the raw tsv data file 
#the file has to be correctly formatted with columns headers  
zeolite_fileObj = open(zeolite_fname)

In [4]:
#create an instance to start processing the datafile
getZeo = GetZeoliteTsv(zeolite_fileObj)

In [5]:
#Sanity check of datatypes
#important to recognise that datatypes are detected from the files
#this step alsos makes the string variables as categorical variables
getZeo.set_dtypes()

,Adsorbent,SA,Vmicro,Vmeso,pore_size,Si_Al,m1,m2,m3,C1,...,Ri3,adsorbate,dipole_moment,chemical_hardness,kinetic_diameter,C_0,solvent,oil_adsorbent_ratio,Temp,Capacity
0,CuAgY,591.00,0.295,NaN,NaN,2.43,Na+,Ag+,Cu+,0.020,...,0.77,TP,0.57,3.0401,0.77,291,cyclohexane,125,50,16.0
1,CuAgY,591.00,0.295,NaN,NaN,2.43,Na+,Ag+,Cu+,0.020,...,0.77,TP,0.57,3.0401,0.77,420,cyclohexane,125,50,24.0
2,CuAgY,591.00,0.295,NaN,NaN,2.43,Na+,Ag+,Cu+,0.020,...,0.77,TP,0.57,3.0401,0.77,556,cyclohexane,125,50,31.0
3,CuAgY,591.00,0.295,NaN,NaN,2.43,Na+,Ag+,Cu+,0.020,...,0.77,TP,0.57,3.0401,0.77,719,cyclohexane,125,50,34.3
4,CuAgY,591.00,0.295,NaN,NaN,2.43,Na+,Ag+,Cu+,0.020,...,0.77,TP,0.57,3.0401,0.77,833,cyclohexane,125,50,35.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,CuHY,399.29,0.190,NaN,NaN,3.40,Na+,NaN,Cu+,0.357,...,0.77,TP,0.51,3.0401,0.77,1100,cyclohexane,20,25,9.8
352,CuHY,399.29,0.190,NaN,NaN,3.40,Na+,NaN,Cu+,0.357,...,0.77,TP,0.51,3.0401,0.77,1219,cyclohexane,20,25,10.5
353,CuHY,399.29,0.190,NaN,NaN,3.40,Na+,NaN,Cu+,0.357,...,0.77,TP,0.51,3.0401,0.77,1321,cyclohexane,20,25,10.9
354,CuHY,399.29,0.190,NaN,NaN,3.40,Na+,NaN,Cu+,0.357,...,0.77,TP,0.51,3.0401,0.77,1418,cyclohexane,20,25,11.2


In [6]:
#this counts the missing records per column and saves them to provided filename
getZeo.missingness("ZeoX_Final_encoded.miss")

PermissionError: [Errno 13] Permission denied: 'ZeoX_Final_encoded.miss'

In [ ]:
#take note of number of columns
getZeo.zeolite_df.shape

In [ ]:
#Drops empty columns inplace
getZeo.zeolite_df.dropna(how='all', axis=1, inplace = True)

In [ ]:
#Very that columns have indeed been lost
getZeo.zeolite_df.shape

In [ ]:
getZeo.missingness("ZeoX_Final_encoded.miss")

In [ ]:
getZeo.GroupMeanImputation('Adsorbent','Vmicro')
getZeo.MeanImputation('Vmicro')

In [ ]:
getZeo.GroupMeanImputation('Adsorbent','Vmeso')
getZeo.MeanImputation('Vmeso')

In [ ]:
getZeo.GroupMeanImputation('Adsorbent','pore_size')
getZeo.MeanImputation('pore_size')

In [ ]:
getZeo.GroupMeanImputation('Adsorbent','pore_size')
getZeo.MeanImputation('pore_size')

In [ ]:
#Check categorical variables
getZeo.df_dtypes

In [ ]:
#convert the categorical variables to intergers also known as one-hot-encoding
#https://towardsdatascience.com/the-dummys-guide-to-creating-dummy-variables-f21faddb1d40
getZeo.encode_categorical()

In [ ]:
for metal in ['C1', 'C2', 'C3', 'x1', 'x2', 'x3', 'Ri1', 'Ri2', 'Ri3']:
         getZeo.zerofill(metal)

In [ ]:
#save the new data to a tsv file
getZeo.save_zeo("ZeoX_Final_encoded.tsv")

In [ ]:
#get our dataframe 
zeolite_final  = getZeo.zeolite_df

In [ ]:
#check our dataframe
zeolite_final.shape

In [ ]:
#We extract our data features 
#attributes 
y = zeolite_final.loc[:,"Capacity"].values
#labels
X = zeolite_final.drop(["Capacity"], axis = 1).values

In [ ]:
#Split our data into training and test dataset 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
#Standardize features by removing the mean and scaling to unit variasnce
sc = StandardScaler()
#https://datascience.stackexchange.com/questions/12321/whats-the-difference-between-fit-and-fit-transform-in-scikit-learn-models#:~:text=%22fit%22%20computes%20the%20mean%20and,both%20at%20the%20same%20time.
#This should not make much of a difference but its good practice
#TO DO
#Compare accuracy with and without scaling
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
n_trees = 10
n_feat = 31
#max_features=n_features,
regressor = RandomForestRegressor(n_estimators = 10, min_samples_split = 2, min_samples_leaf = 1, max_features = 37, max_depth = 28, random_state=1000)
#TO DO
#increase n_estimators
#run in parallel

In [ ]:
regressor.fit(X_train, y_train)

In [ ]:
regressor.get_params()

In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:
plot_data = pd.DataFrame.from_dict({'y_pred': y_pred, 'y_test': y_test, 'errors': y_pred - y_test, 'abs_errors': abs(y_pred - y_test)})
plot_data.to_csv("RF_model_performance.tsv", sep = "\t", index = False)

In [ ]:
pd.options.display.max_rows = 4000

In [ ]:
mae = metrics.mean_absolute_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
rmse = metrics.mean_squared_error(y_test, y_pred, squared = False)
mape = metrics.mean_absolute_percentage_error(y_test, y_pred)
r2 =  metrics.r2_score(y_test, y_pred)

In [ ]:
data_table = pd.DataFrame.from_dict({"n_feat": [n_feat],
                                    "n_trees":[n_trees],
                                     "mae": [mae], 
                                     "mse": [mse], 
                                     "rmse":[rmse],
                                     "r2":[r2],
                                     "mape":[mape]})

In [ ]:
data_table

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(y_test, y_pred)

In [ ]:
print("Correlation coefficient (R): {:.4f} ".format(r_value))
print("p-value : {}".format(p_value))
print("Intercept: {:.4f}".format(intercept))
print("Slope: {:.4f}".format(slope))
print("std_error: {:.4f}".format(std_err))

In [ ]:
ax = sns.regplot(y="y_pred",
                 x="y_test", 
                 color="g", 
                 marker="+",
                 line_kws={'label':'$r^2$ = {:.2f}'.format(r_value**2)},
                 data = plot_data)

plt.ylabel('Predicted adsorptive capacity (mgS/g)')
plt.xlabel('Experimental adsorptive capacity (mgS/g)')
ax.legend(loc=9)
plt.savefig('traning_r2.pdf', format='pdf', dpi=1200)
plt.show()
os.getcwd()

In [ ]:
feature_importance = pd.DataFrame(data = {"features":zeolite_final.drop(["Capacity"], axis = 1).columns, "importance":regressor.feature_importances_} )
feature_importance.to_csv("RFX_feature_importance.tsv", sep = "\t", index = False)

In [ ]:
?regressor.feature_importances_

In [ ]:
feature_importance["importance"] = round(feature_importance["importance"],5)

In [ ]:
sum(feature_importance.importance)

In [ ]:
feature_importance.sort_values("importance", ascending=False)

In [ ]:
result = permutation_importance(regressor, X_train, y_train)

In [ ]:
#result.importances_mean

In [ ]:
feature_importance_PI = pd.DataFrame(data = {"features":zeolite_final.drop(["Capacity"], axis = 1).columns, 
                                          "importance_mean":result.importances_mean,
                                          "importance_std": result.importances_std} )
feature_importance_PI.to_csv("RF_feature_importance_PI.tsv", sep = "\t", index = False)

In [ ]:
?result.importances_mean

In [ ]:
feature_importance_PI["importance_mean"] = round(feature_importance_PI["importance_mean"], 5)

In [ ]:
feature_importance_PI.sort_values("importance_mean", ascending=False)

In [ ]:
#https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
# Number of trees in random forest
n_estimators = list(range(1,20))
# Number of features to consider at every split
max_features = list(range(1,40))
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(20, 40, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [1, 2, 4]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree

In [ ]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

In [ ]:
random_grid

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 10, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

In [ ]:
rf_random.best_params_

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [ ]:
base_model = RandomForestRegressor(n_estimators=n_trees, max_features=n_feat, random_state=42)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)

In [ ]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_train, y_train)